In [1]:
import torch
import os
import sys
sys.path.append('lib/model_gen/')
sys.path.append('../xf_prelims/')
import xf_model_training as xmt
import merge_consensus
import pandas as pd
import pod5
import multiprocessing
import numpy as np
from pathlib import Path

In [2]:
wdir = '/media/sebastian/Slepnir/xenofind_datasets/Working_directory_PZ/'
rdir = '/media/sebastian/Slepnir/xenofind_datasets/PZ_dataset/231117_PZn_libv4_minion/20231117_1644_MN37138_FAX70078_76df2751/fast5'
refdir = '/media/sebastian/Slepnir/xenofind_datasets/PZ_dataset/231117_PZn_libv4_minion/fasta_metadata/ref_libv2_PZ_CxDx.fa'
#mp5_path, al_bam_path = xmt.preprocessing(wdir, rdir, refdir)

In [3]:
#pz_aligned_bam = merge_consensus.load_in_data('/media/sebastian/Slepnir/xenofind_datasets/Working_directory_PZ/model_training/basecall_directory/basecall.bam')
p5 = '/home/sebastian/Documents/GitHub/XenoFind/data/large_working_directory/merged_pod5/merged.pod5'
p5_max = '/media/sebastian/Slepnir/xenofind_datasets/Working_directory_PZ/model_training/merged_pod5/merged.pod5'
bam = "/home/sebastian/Documents/GitHub/XenoFind/data/large_working_directory/rough_consensus_output/large_align.bam"
pz_aligned_bam = merge_consensus.load_in_data(bam)

In [17]:
bam_seq_ids = {x['seq_id'] for x in pz_aligned_bam}

In [6]:
bam_ref_ids = {x['ref_name'] for x in pz_aligned_bam}

In [4]:

class Read:
    def __init__(self, rid, sig, freq):
        self.read_id = rid
        self.sig = np.array(list(sig), dtype=np.float32)
        self.freq = freq
        
def iterable_reads(p5_path, valid_reads):
    with pod5.Reader(Path(p5_path)) as pod5_file:
        for read in pod5_file.reads(selection=valid_reads, preload=["samples"]):
            out_read = Read(read.read_id, read.signal_pa, read.run_info.sample_rate)
            yield out_read
            

In [5]:
class JointBamReads:
    def __init__(self, ref_name, readiterator):
        self.ref_name = ref_name
        self.read_iterator = readiterator

In [10]:
with pod5.Reader(Path(p5)) as pod5_file:
    pod5_read_ids = set(pod5_file.read_ids)

aggregate_refs = []
bam_ref_ids = [x['ref_name'] for x in pz_aligned_bam]
for unique_ref in bam_ref_ids[:20]:
    print(unique_ref, end='\r')
    temp_df = pd.DataFrame(pz_aligned_bam)
    seqs = set(temp_df[temp_df['ref_name'] == unique_ref]['seq_id'])
    valid_reads_in_ref = list(pod5_read_ids.intersection(seqs))
    aggregate_refs.append(JointBamReads(unique_ref, iterable_reads(p5, valid_reads_in_ref)))
    

In [21]:
pd.DataFrame(pz_aligned_bam)[pd.DataFrame(pz_aligned_bam)['seq_id']=='2a11fb72-4523-4fd8-83ab-68366cd8dac3']

,seq_id,ref_name,sequence,quality,cigar,len,ref,rev,moves,sig_len,trim_ofs
145238,2a11fb72-4523-4fd8-83ab-68366cd8dac3,Pool=ACGTAACTTGGTTTGTTCCCTGAA-AGTCAGCT+CGAGGAG...,TGAAGATCAGTCAGCTACTGCGACGGCCTGAGCAGTTTCCCTGCCG...,"[4, 4, 9, 9, 9, 10, 10, 11, 9, 9, 9, 9, 13, 13...","[(4, 9), (0, 21), (2, 1), (0, 2), (2, 2), (0, ...",94,39,True,"[5, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, ...",1524,10


In [ ]:
class AggregateRefData:
    def __init__(self, )

In [22]:
for ref in (aggregate_refs):
    for read in ref.read_iterator:
        tmp_df = pd.DataFrame(pz_aligned_bam)
        bam_read_info = tmp_df[tmp_df['seq_id'] == str(read.read_id)]
        

                                      seq_id  \
165008  919e7400-eee7-4822-b32d-fc5b07da0c28   

                                                 ref_name  \
165008  Pool=ACGTAACTTGGTTTGTTCCCTGAA-AGTCAGCT+CGAGGAG...   

                                                 sequence  \
165008  AGTACTAATTTTTTTCGATACGTAACTTGGTTTGTTTGCTGAAGAT...   

                                                  quality  \
165008  [35, 34, 33, 33, 30, 30, 30, 31, 31, 32, 32, 3...   

                                                    cigar  len  ref    rev  \
165008  [(4, 40), (0, 61), (1, 1), (0, 10), (1, 1), (0...  123    0  False   

                                                    moves  sig_len  trim_ofs  
165008  [5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...     1875        10  
                                      seq_id  \
165699  36b60cab-c577-4ac1-9b57-2bd38b15e8c7   

                                                 ref_name  \
165699  Pool=ACGTAACTTGGTTTGTTCCCTGAA-AGTCAGCT+CGAGGAG...   


KeyboardInterrupt: 

In [25]:
from pathlib import Path
with pod5.Reader(Path(p5)) as pod5_file:
    pod5_read_ids = set(pod5_file.read_ids)
# Filter the pod5 read ids to those that are in the bam ids
v_reads = list(pod5_read_ids.intersection(bam_seq_ids))

class Read:
    def __init__(self, rid, sig, freq):
        self.read_id = rid
        self.sig = np.array(list(sig), dtype=np.float32)
        self.freq = freq
        
def iterable_reads(p5_path, valid_reads):
    with pod5.Reader(Path(p5_path)) as pod5_file:
        for read in pod5_file.reads(selection=valid_reads, preload=["samples"]):
            out_read = Read(read.read_id, read.signal_pa, read.run_info.sample_rate)
            yield out_read
            
readiterator = iterable_reads(p5, v_reads)

KeyboardInterrupt: 

In [1]:
p5_path = "/media/sebastian/Slepnir/xenofind_datasets/Working_directory_PZ/model_training/merged_pod5/merged.pod5"
import pod5_extractor_methods
import pandas as pd
import sys

In [2]:
data_dict = pod5_extractor_methods.extract_pod5_dict(p5_path, 20000, 3)

on 3: 3/6
on 3: [ Memory Report 2024-05-17 20:20:42.083597 ]------------------------------------------------
on 3: RSS: 1.049034752 Gb, VMS: 4.116926464 Gb, SHARED: 0.057638912 Gb, DATA: 1.860947968 Gb
on 3: --------------------------------------------------------------------------------------------
on 6: 6/6
on 6: [ Memory Report 2024-05-17 20:20:58.579919 ]------------------------------------------------
on 6: RSS: 2.18232832 Gb, VMS: 5.235965952 Gb, SHARED: 0.057638912 Gb, DATA: 2.994057216 Gb
on 6: --------------------------------------------------------------------------------------------
|████████████████████████████████████████| 6/6 [100%] in 28.3s (0.21/s) 


In [3]:
len(data_dict)

559972

In [3]:
pd.DataFrame(data_dict).to_json('reads.json')

In [ ]:
pd.read_json('reads.json')

In [12]:
(pd.DataFrame(data_dict)).memory_usage()/(1*10**9)

Index    1.280000e-07
0        4.479776e-03
1        4.479776e-03
2        1.119944e-03
dtype: float64

In [7]:

np.int64(120.39091)

120